In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
!ollama pull llama3.2

from openai import OpenAI
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [3]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
site = Website("https://www.tripadvisor.com/")
site.links

['/Trust-lgF5hKLTqw3U-Accessibility_statement.html',
 '/',
 '/RegistrationController?flow=sign_up_and_save&returnTo=%2F&fullscreen=true&flowOrigin=login&hideNavigation=true&isLithium=true',
 '/Hotels',
 '/Attractions',
 '/Restaurants',
 '/CheapFlightsHome',
 '/Rentals',
 '/TripBuilder',
 '/TravelersChoice-Beaches-cWorld-g1',
 '/TravelersChoice-Beaches-cEurope-g4',
 '/TravelersChoice-Beaches-cAsia-g2',
 '/TravelersChoice-Beaches-cSouthPacific-g8',
 '/TravelersChoice-Beaches-cCaribbean-g147237',
 '/AttractionProductReview-g189180-d25382579-The_Unvanquished_Tour_in_Porto_City_Center-Porto_Porto_District_Northern_Portugal.html',
 '/AttractionProductReview-g297701-d15142407-All_Inclusive_Ubud_Private_Tour-Ubud_Gianyar_Regency_Bali.html',
 '/AttractionProductReview-g188590-d17809082-All_Inclusive_Amsterdam_Canal_Cruise_with_Drinks_and_Dutch_Bites-Amsterdam_North_H.html',
 '/AttractionProductReview-g297390-d14080946-Small_Group_Explore_Angkor_Wat_Sunrise_Tour_with_Guide_from_Siem_Reap-Siem_Re

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(site))

Here is the list of links on the website of https://www.tripadvisor.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/Trust-lgF5hKLTqw3U-Accessibility_statement.html
/
/RegistrationController?flow=sign_up_and_save&returnTo=%2F&fullscreen=true&flowOrigin=login&hideNavigation=true&isLithium=true
/Hotels
/Attractions
/Restaurants
/CheapFlightsHome
/Rentals
/TripBuilder
/TravelersChoice-Beaches-cWorld-g1
/TravelersChoice-Beaches-cEurope-g4
/TravelersChoice-Beaches-cAsia-g2
/TravelersChoice-Beaches-cSouthPacific-g8
/TravelersChoice-Beaches-cCaribbean-g147237
/AttractionProductReview-g189180-d25382579-The_Unvanquished_Tour_in_Porto_City_Center-Porto_Porto_District_Northern_Portugal.html
/AttractionProductReview-g297701-d15142407-All_Inclusive_Ubud_Private_Tour-Ubud_Gianyar_Regency_Bali.html
/AttractionProductR

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://www.tripadvisor.com/")

{'links': [{'type': 'About us page',
   'url': 'https://tripadvisor.mediaroom.com/us-about-us'},
  {'type': 'Company Overview',
   'url': 'https://tripadvisor.mediaroom.com/us-contact-us'},
  {'type': 'Resources/FAQs',
   'url': 'https://tripadvisor.mediaroom.com/us-resources'},
  {'type': 'Careers page', 'url': 'https://careers.tripadvisor.com'}]}

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [45]:
print(get_all_details("https://www.tripadvisor.com/"))

Found links: {'links': [{'type': 'About page', 'url': 'https://tripadvisor.mediaroom.com/us-about-us'}, {'type': 'Contact us', 'url': 'https://tripadvisor.mediaroom.com/us-contact-us'}, {'type': 'Resource page', 'url': 'https://tripadvisor.mediaroom.com/us-resources'}, {'type': 'Careers/jobs', 'url': 'https://careers.tripadvisor.com'}, {'type': 'Investor relations/IR', 'url': 'http://ir.tripadvisor.com'}]}


In [12]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [14]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [16]:
create_brochure("Ed", "https://edwarddonner.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'Company page', 'url': 'https://edwarddonner.com/'}]}


**Ed Brochure**
===============

**About Us**
-------------

Ed is the brainchild of Ed Donner, a passionate AI enthusiast and entrepreneur. With a long-term goal to help people discover their potential and pursue their reason for being, Ed aims to revolutionize the field of talent acquisition using Generative AI.

**Our Mission**
----------------

At Ed, we believe that everyone deserves to find their dream job. Our patented model matches people with roles with greater accuracy and speed than previously imaginable, without requiring specific keywords. We're on a mission to raise the level of human prosperity by helping people discover their Ikigai (purpose in life).

**Our Story**
--------------

Ed's journey began when Ed founded AI startup untapt in 2013, specializing in tech roles for financial services. After being part of a prestigious accelerator program and being recognized as an American Banker Top 20 Company To Watch, untapt was acquired by GQR's parent company in 2021. Shortly after, Nebula.io was born.

**Our Product**
----------------

Ed's flagship product is the award-winning platform that uses groundbreaking, proprietary LLLMs (Large Language Models) to source, understand, engage and manage talent. Our matching model has been patented and has happy customers and tons of press coverage.

**Connect with Us**
------------------

Stay in touch with Ed through our website, social media channels, or by subscribing to our newsletter. Follow us on LinkedIn, Twitter, Facebook, or send an email to [ed@edwarddonner.com](mailto:ed@edwarddonner.com). Let's connect and discuss how we can revolutionize talent acquisition together!

**Join Our Community**
----------------------

Are you a recruiter looking for innovative solutions? A software engineer interested in AI? Or an entrepreneur seeking mentorship? Connect with us for virtual coffee or real-world collaboration. Let's work together to change the face of talent acquisition.

**Stay Inspired**
------------------

Inspiring stories and industry insights from around the world. Join our community to stay up-to-date on the latest developments and thought leadership in AI, recruitment, and entrepreneurship.

In [17]:
import gradio as gr

In [18]:
view = gr.Interface(
    fn= create_brochure,
    inputs=[
        gr.Textbox(label="Company Name:"),
        gr.Textbox(label="Company Url:"),
        ],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


Found links: {'links': [{'type': 'About page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'Homepage', 'url': 'https://edwarddonner.com/'}]}


**Eddd Brochure**
======================

**Who We Are**
---------------

Meet Eddd, a passionate team of innovators dedicated to helping people discover their potential and pursue their reason for being. Our mission is to empower individuals by applying AI in a way that can make a massive, positive impact.

**Our Story**
-------------

Founded by Ed Donner, our CEO and CTO, who has spent 20 years exploring the field of AI and its real-world applications. Initially, he co-founded an AI startup called untapt in 2013, which specialized in talent marketplaces and data science software for recruitment firms.

**Experience**
------------

* Led a team that built award-winning platforms using proprietary LLMs
* Patented matching models for talent sourcing
* Partnered with top clients and gained press coverage in prominent industry publications
* Joined Nebula.io after untapt's acquisition by GQR in 2021, expanding our focus to source, understand, engage, and manage talent

**Our Focus**
-------------

Nebula.io helps recruiters find the most talented candidates, ensuring they're connected with roles where they'll be inspired and engaged. Our cutting-edge platform uses AI to match people with their optimal fit, without requiring keywords.

**Why We Care**
----------------

We believe in making a difference in people's lives by helping them discover their passions and pursue careers that fulfill them. This approach is driven by our concept of Ikigai, which highlights the importance of finding roles where human vitality and happiness come together.

**Our Current Arena: Connect Four**
---------------------------------

Imagine an arena where LLMs compete against each other in a battle of diplomacy and cunning. At Eddd, we're developing this "Connect Four" challenge to push AI boundaries and promote human-technology collaboration.

**We'd Love to Connect With You!**
------------------------------

Visit our website or send us a message through Ed's contact email. Join the conversation on LinkedIn, Twitter, or Facebook to explore how your interests align with Eddd's mission.

Stay tuned for updates from our team, resources on AI and LLM engineering, and insights into improving talent sourcing in the digital age.